In [1]:
import numpy as np
from torch.utils.data import DataLoader
from torch import nn
import random
import math
import torch
from torch import nn
from torch import optim
from tensorboardX import SummaryWriter
from datetime import datetime
import time
import load
from models import DynamicTopologyModel
import load_topology_dataset
from load_topology_dataset import MAX_CELL_COUNT, MIN_CELL_COUNT, IMAGE_SIZE
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device="cpu"
EXPERIMENT=1
writer = SummaryWriter('runs/experiment_{}/'.format(EXPERIMENT))

In [2]:
def save_parameters(writer, model, batch_idx):
    for k, v in model.state_dict().items():
        shape = v.shape
        # Don't do this for single weights or biases
        if np.any(np.array(shape) > 1):
            mean = torch.mean(v)
            std_dev = torch.std(v)
            maximum = torch.max(v)
            minimum = torch.min(v)
            writer.add_scalars("{}_{} ".format(k, shape), {"mean": mean,
                                                                    "std_dev": std_dev,
                                                                    "max": maximum,
                                                                    "min": minimum}, batch_idx)
            writer.add_histogram("{}_{}".format(k, shape), v, batch_idx)
            
        else:
            writer.add_scalar("{}_{}".format(k, shape), v.data, batch_idx)

In [3]:
def test_parameters_update(last_parameters_np ,parameters_np):
    for last_param, param in zip(last_parameters_np, parameters_np):
        diff = param - last_param
        #print(last_parameters.max())
        print(diff.max())
        #print(not np.all(diff))


In [4]:
def print_example(reference_cell_input, reference_cell_present_input,
                neighbourhood_cell_rel_input, neighbourhood_cell_load_input, neighbourhood_cell_present_input,
                  reference_cell_target, reference_cell_present_target, neighbourhood_cell_rel_target,
                  neighbourhood_cell_present_target):
    print("reference_cell_input") 
    print(reference_cell_input)
    print("reference_cell_present_input") 
    print(reference_cell_present_input)
    print("neighbourhood_cell_rel_input") 
    print(neighbourhood_cell_rel_input)
    print("neighbourhood_cell_load_input") 
    print(neighbourhood_cell_load_input)
    print("neighbourhood_cell_present_input") 
    print(neighbourhood_cell_present_input)
    print("reference_cell_target") 
    print(reference_cell_target)
    print("reference_cell_present_target") 
    print(reference_cell_present_target)
    print("neighbourhood_cell_rel_target") 
    print(neighbourhood_cell_rel_target)
    print("neighbourhood_cell_present_target") 
    print(neighbourhood_cell_present_target)

In [5]:

# BATCH SIZE STILL NOT WORKING!!!

BATCH_SIZE = 8 
INPUT_SEQ_LEN = 8 
TARGET_SEQ_LEN = 8
LEARNING_RATE = 1e-3
SAVE_VARIABLE_EVERY = 100
DATASET_WORKERS = 2


input_seq_len= torch.Tensor([INPUT_SEQ_LEN]).int()
target_seq_len= torch.Tensor([TARGET_SEQ_LEN]).int()

# Define Data.
cell_load_dataset = load_topology_dataset.LoadCellDataset(initial_cell_counts=(2, 5), initial_load_counts=(2,5),
                                    input_seq_len=INPUT_SEQ_LEN, target_seq_len=TARGET_SEQ_LEN, network_mutate_prob=[0.5, 0.5
                                                                                                                    ])
dataloader = DataLoader(cell_load_dataset, batch_size=BATCH_SIZE,
                        shuffle=False, num_workers=DATASET_WORKERS)

# Define Model
dynamic_topology_model = DynamicTopologyModel(neighbourhood_hidden_size=32, neighbourhood_cell_count=6,
                     neighbourhood_output_size=16, lstm_hidden_size=32, lstm_layers=2, teacher_forcing_probability=0.5, device=device).to(device)

parameters = dynamic_topology_model.parameters

# Define optimizer
optimizer = optim.Adam(parameters
                       , lr=LEARNING_RATE)
criterion = nn.MSELoss()
last_parameters_np = list(np.copy(param.cpu().detach().numpy()) for param in parameters)

# Train
start = datetime.now()
for batch_idx, data in enumerate(dataloader):
    optimizer.zero_grad()
    reference_cell_input =  data[0].to(device)
    reference_cell_present_input = data[1].to(device)
    neighbourhood_cell_rel_input = data[2].to(device)
    neighbourhood_cell_load_input = data[3].to(device)
    neighbourhood_cell_present_input = data[4].to(device)
    reference_cell_target = data[5].to(device)
    reference_cell_present_target = data[6].to(device)
    neighbourhood_cell_rel_target = data[7].to(device)
    neighbourhood_cell_present_target = data[8].to(device)
    
    #print_example(reference_cell_input, reference_cell_present_input,
    #            neighbourhood_cell_rel_input, neighbourhood_cell_load_input, neighbourhood_cell_present_input,
    #              reference_cell_target, reference_cell_present_target, neighbourhood_cell_rel_target,
    #              neighbourhood_cell_present_target)    
    
    decoder_output_seq = dynamic_topology_model.forward(input_seq_len, target_seq_len, reference_cell_input, reference_cell_present_input,
                neighbourhood_cell_rel_input, neighbourhood_cell_load_input, reference_cell_target, reference_cell_present_target, neighbourhood_cell_rel_target)
    
    loss = criterion(decoder_output_seq, reference_cell_target)
    loss.backward()
    optimizer.step()
    if batch_idx == 0:
        parameters = dynamic_topology_model.parameters
        parameters_np = list(param.cpu().detach().numpy() for param in parameters)
        test_parameters_update(last_parameters_np, parameters_np)
        inputs =(input_seq_len, target_seq_len, reference_cell_input, reference_cell_present_input,
                neighbourhood_cell_rel_input, neighbourhood_cell_load_input, reference_cell_target, reference_cell_present_target, neighbourhood_cell_rel_target)
        writer.add_graph(dynamic_topology_model, inputs, True)
    if batch_idx % SAVE_VARIABLE_EVERY == 0:
        end = datetime.now()
        print("{} Examples/sec".format((SAVE_VARIABLE_EVERY* BATCH_SIZE) / ((end - start).total_seconds()) ))
        start = datetime.now()
        print("loss: {}".format(loss))
        #print("output: {}".format(decoder_output_seq))
        #print("target: {}".format(reference_cell_target))
        writer.add_scalar("loss", loss, batch_idx)
        save_parameters(writer, dynamic_topology_model, batch_idx)


0.0009558499
0.0009992719
0.0009872317
0.0009997487
0.0009997934
0.0009998679
0.0009999573
0.0009999555
0.0009999871
0.0
0.0009999461
0.0009999424
0.0009999424
0.0
0.0009999871
0.000999988
0.0
0.0
0.0009994209
0.0009999871
0.001000002
0.001000002
0.001000002
0.0009999871
0.0009999946
0.0009999946
0.001000002
0.0009999946
0.001000002
0.001000002
0.001000002
0.001000002


/Users/d854956/personal/dynamic_topology/models.py:196: TracerWarning: Converting a tensor to a NumPy array might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  (batch_size, target_seq_len.data.numpy()[0], 1), device=self.device
/Users/d854956/personal/dynamic_topology/models.py:200: TracerWarning: Converting a tensor to a Python index might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  for input_seq_idx in range(input_seq_len):
/Users/d854956/personal/dynamic_topology/models.py:231: TracerWarning: Converting a tensor to a Python index might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the f

graph(%0 : Int(1)
      %1 : Int(1)
      %2 : Float(8, 8, 1)
      %3 : Float(8, 8, 1)
      %4 : Float(8, 8, 6, 2)
      %5 : Float(8, 8, 6, 1)
      %6 : Float(8, 8, 1)
      %7 : Float(8, 8, 1)
      %8 : Float(8, 8, 6, 2)
      %9 : Float(32, 2)
      %10 : Float(32)
      %11 : Float(32, 1)
      %12 : Float(32)
      %13 : Float(1, 32)
      %14 : Float(1)
      %15 : Float(1, 32)
      %16 : Float(1)
      %17 : Float(128, 3)
      %18 : Float(128, 32)
      %19 : Float(128)
      %20 : Float(128)
      %21 : Float(128, 32)
      %22 : Float(128, 32)
      %23 : Float(128)
      %24 : Float(128)
      %25 : Float(1, 32)
      %26 : Float(1)
      %27 : Float(32, 2)
      %28 : Float(32)
      %29 : Float(16, 32)
      %30 : Float(16)
      %31 : Float(128, 18)
      %32 : Float(128, 32)
      %33 : Float(128)
      %34 : Float(128)
      %35 : Float(128, 32)
      %36 : Float(128, 32)
      %37 : Float(128)
      %38 : Float(128)
      %39 : Float(1, 32)
      %40 : Float(1)) {

KeyboardInterrupt: 

In [ ]:
PIXELS = 640
LEFT = 1
RIGHT = 3
INPUT_SCREEN_COUNT = 1
TARGET_SCREEN_COUNT = 1
SCREEN_COUNT = INPUT_SCREEN_COUNT + TARGET_SCREEN_COUNT

PIXELS_PER_BLOCK = int(PIXELS / IMAGE_SIZE)
SCALE_ARR = np.ones((PIXELS_PER_BLOCK, PIXELS_PER_BLOCK, 1))

load_screen = np.zeros((IMAGE_SIZE, IMAGE_SIZE, 1))
load_screen[1,12, 0] = 255
load_screen[22,22, 0] = 255
loads = [(1, 12), (22, 22)]
block_img = np.kron(load_screen, SCALE_ARR)

screen = pygame.display.set_mode((PIXELS, PIXELS))
screens = []

screen_draw_idx = 0
draw_on = False
last_pos = (0, 0)
color = (255, 128, 0)
radius = 10
block_screen = np.zeros((IMAGE_SIZE, IMAGE_SIZE, 3))
model_screen = np.zeros((IMAGE_SIZE, IMAGE_SIZE, 3), dtype=np.uint8)
frame = 0

In [ ]:
   model_target = [list() for i in range(TARGET_SCREEN_COUNT)]
    for reference_cell in cells_seq[-1]:
        #print(reference_cell)
        ref_x, ref_y = reference_cell[0], reference_cell[1]
        reference_cell_input = np.zeros((INPUT_SCREEN_COUNT))
        reference_cell_present_input = np.zeros((INPUT_SCREEN_COUNT))
        neighbourhood_cell_rel_input = np.zeros((INPUT_SCREEN_COUNT, MAX_CELL_COUNT, 2))
        neighbourhood_cell_load_input = np.zeros((INPUT_SCREEN_COUNT, MAX_CELL_COUNT))
        neighbourhood_cell_present_input = np.zeros((INPUT_SCREEN_COUNT, MAX_CELL_COUNT))
        for seq_idx in range(INPUT_SCREEN_COUNT):
            reference_cell_active = False
            cell_idx = 0
            for cell in load_cells_seq_input[seq_idx]:
                current_x = cell[0]
                current_y = cell[1]
                current_load = cell[2]
                # If not reference cell add to neighbourhood
                if not (current_x == ref_x and current_y == ref_y):
                    # neighbourhood normalized by dividing by IMAGE_SIZE.
                    # Subtract from 1 since cells that are closes have greated influence.
                    #print("seq_idx {}".format(seq_idx))
                    #print("cell_idx {}".format(cell_idx))
                    neighbourhood_cell_rel_input[seq_idx, cell_idx, 0] = 1. - abs(ref_x - current_x) / IMAGE_SIZE
                    neighbourhood_cell_rel_input[seq_idx, cell_idx, 1] = 1. - abs(ref_y - current_y) / IMAGE_SIZE
                    neighbourhood_cell_load_input[seq_idx, cell_idx] = current_load
                    cell_idx += 1
                else:
                    # print("ref cell")
                    reference_cell_input[seq_idx] = current_load
                    reference_cell_active = True
            if reference_cell_active:
                reference_cell_present_input[seq_idx] = 1.0

        load_cells_seq_target = cells_load_seq[INPUT_SCREEN_COUNT:]
        reference_cell_present_target = np.zeros((TARGET_SCREEN_COUNT))
        neighbourhood_cell_rel_target = np.zeros((TARGET_SCREEN_COUNT, MAX_CELL_COUNT, 2))


        for seq_idx in range(TARGET_SCREEN_COUNT):
            reference_cell_active = False
            cell_idx = 0
            for cell in load_cells_seq_target[seq_idx]:
                current_x = cell[0]
                current_y = cell[1]
                current_load = cell[2]
                #print(current_load)
                # If not reference cell add to neighbourhood
                if not (current_x == ref_x and current_y == ref_y):
                    # neighbourhood normalized by dividing by IMAGE_SIZE.
                    # Subtract from 1 since cells that are closes have greated influence.
                    neighbourhood_cell_rel_target[seq_idx, cell_idx, 0] = 1. - abs(ref_x - current_x) / IMAGE_SIZE
                    neighbourhood_cell_rel_target[seq_idx, cell_idx, 1] = 1. - abs(ref_y - current_y) / IMAGE_SIZE
                    cell_idx += 1
                else:
                    reference_cell_active = True
            if reference_cell_active:
                reference_cell_present_target[seq_idx] = 1.0

        reference_cell_input = torch.tensor(reference_cell_input, dtype=torch.float32).unsqueeze(0)
        reference_cell_present_input = torch.tensor(reference_cell_present_input, dtype=torch.float32).unsqueeze(0)
        neighbourhood_cell_rel_input = torch.tensor(neighbourhood_cell_rel_input, dtype=torch.float32).unsqueeze(0)
        neighbourhood_cell_load_input = torch.tensor(neighbourhood_cell_load_input, dtype=torch.float32).unsqueeze(0)
        reference_cell_present_target = torch.tensor(reference_cell_present_target, dtype=torch.float32).unsqueeze(0)
        neighbourhood_cell_rel_target = torch.tensor(neighbourhood_cell_rel_target, dtype=torch.float32).unsqueeze(0)


        
        
        
        model_target[seq_idx].append((ref_x, ref_y, decoder_output.detach().numpy()[0][0][0]))

In [ ]:
 0
cells_seq = [list() for i in range(SCREEN_COUNT)]
cells_load_seq = [list() for i in range(SCREEN_COUNT)]
BATCH_SIZE=1
show_labels = True

while True:
    
    ##############################
    # Use model to forecast future load
    ##############################
    load_cells_seq_input = cells_load_seq[:INPUT_SCREEN_COUNT]
    load_cells_seq_target = cells_load_seq[INPUT_SCREEN_COUNT:]
    
 
            
    ################################
    # DISPLAY
    #################################
    screen_draw = pygame.Surface((PIXELS, PIXELS))
    
    #print(cells_seq)
    cells = cells_seq[screen_draw_idx]
    cell_loads = cells_load_seq[screen_draw_idx]

    if (screen_draw_idx < INPUT_SCREEN_COUNT) or (show_labels and screen_draw_idx >= INPUT_SCREEN_COUNT):
        for cell in cell_loads:
            color = (cell[2] * 1024) - 256
            if color > 255:
                color = 255
            if color < 0:
                color = 0
            pygame.draw.rect(
            screen_draw,
            (color, 50, 0),
            (
                cell[0] * PIXELS_PER_BLOCK,
                cell[1] * PIXELS_PER_BLOCK,
                PIXELS_PER_BLOCK,
                PIXELS_PER_BLOCK,
            ),
            )
    else:
        model_loads = model_target[screen_draw_idx - INPUT_SCREEN_COUNT]
        for cell in model_loads:
            color = (cell[2] * 1024) - 256
            if color > 255:
                color = 255
            if color < 0:
                color = 0
            pygame.draw.rect(
            screen_draw,
            (color, 50, 0),
            (
                cell[0] * PIXELS_PER_BLOCK,
                cell[1] * PIXELS_PER_BLOCK,
                PIXELS_PER_BLOCK,
                PIXELS_PER_BLOCK,
            ),
            )

    e = pygame.event.poll()


    if e.type == pygame.QUIT:
        raise StopIteration
    if e.type == pygame.MOUSEBUTTONDOWN:
        leftclick, middleclick, rightclick = pygame.mouse.get_pressed()
        x, y = e.pos
        x_cell= int(x / PIXELS_PER_BLOCK)
        y_cell = int(y / PIXELS_PER_BLOCK)
        if leftclick:
            cells = add_cell_if_not_exists((x_cell, y_cell), cells)
        elif rightclick:
            cells = remove_cell_if_exists((x_cell, y_cell), cells)            
        draw_on = True
        # Update cells
        cells_seq[screen_draw_idx] = cells
        
        # Update all cell loads
        for i, cells in enumerate(cells_seq):
            cells_load_seq[i] = load.calculate_cell_load(cells, loads)
    
    if e.type == pygame.KEYDOWN:
        if e.key == pygame.K_SPACE:
            if show_labels:
                show_labels = False
            else:
                show_labels = True
                print(model_target[-1])
                print(load_cells_seq_target[-1])
        if e.key ==  pygame.K_LEFT:
            if screen_draw_idx > 0:
                screen_draw_idx -= 1
        if e.key ==  pygame.K_RIGHT:
            if screen_draw_idx < SCREEN_COUNT - 1:
                screen_draw_idx += 1
    if e.type == pygame.MOUSEBUTTONUP:
        draw_on = False
    pygame.display.flip()
    target_screen = pygame.surfarray.array3d(screen_draw)
    display_img = block_img + target_screen
    new_surf = pygame.pixelcopy.make_surface(display_img.astype(np.uint8))
    screen.blit(new_surf, (0, 0))